In [15]:
import numpy as np
import matplotlib.pyplot  as plt
from pathlib import Path
import struct

In [16]:
train_img_path = "image\\train-images.idx3-ubyte"
train_lab_path = 'image\\train-labels.idx1-ubyte'
test_img_path = "image\\t10k-images.idx3-ubyte"
test_lab_path = 'image\\t10k-labels.idx1-ubyte'


train_img_f = open(train_img_path,'rb')
train_lab_f = open(train_lab_path,'rb')
test_img_f = open(test_img_path,'rb')
test_lab_f = open(test_lab_path,'rb')

In [17]:
struct.unpack('>4i',train_img_f.read(16))
struct.unpack('>ii',train_lab_f.read(8))
struct.unpack('>4i',test_img_f.read(16))
struct.unpack('>ii',test_lab_f.read(8))

train_imgs = np.fromfile(train_img_f,dtype=np.uint8).reshape(-1,28*28)
train_labs = np.fromfile(train_lab_f,dtype=np.uint8)
test_imgs = np.fromfile(test_img_f,dtype=np.uint8).reshape(-1,28*28)
test_labs = np.fromfile(test_lab_f,dtype=np.uint8)
print(test_labs)

[7 2 1 ... 4 5 6]


In [18]:
############整理train_labs，使其变成 60000*10#################
y = np.zeros((60000,10))
for i in range(y.shape[0]):
    y[i][train_labs[i]]=1
print(y.shape)
print(y[0:10,:])

(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [19]:
def sigmond(x):
    return 1/(1+np.exp(-np.abs(x)))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

In [20]:
def J(hy,y):
   return - np.sum(y*np.log(hy) + (1-y)*np.log(1-y)) / 60000

def J_softmax(h,y):
    b = y*np.log(h)
    return b.sum() / 60000

In [21]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1.txt",delimiter=',')
w2 = np.loadtxt("init_w2.txt",delimiter=',')
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.1
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10,0:5])

(785, 800)
(801, 10)


In [22]:
train_imgs.max()

255

In [23]:
#####################开始训练传播##################
for i in range(1000):
    x = train_imgs - 255
    ######## x.shape = 60000*784 ############
    x = np.insert(x,0,1,axis=1)
    ######## x.shape = 60000*785  ############
    
    ############ 隐藏层输入 ############
    hid_input = np.matmul(x,w1)
    ######## hid_input.shape = 60000*785 ############
    
    ############ 隐藏层输出 ############
    hid_output = sigmond(hid_input)
    ######## hid_output.shape = 60000*392 ############
    
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)
    ######## hid_output.shape = 60000*393 ############
    
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ######## hpfs_input.shape = 60000*10 ############
    
     ############ 最后一层的输出，即正向传播的输出值 ############
    hpfs_output = softmax(hpfs_input)
    #print(hpfs_output.shape)
    ######## hpfs_output.shape = 60000*10 ############
    ############ 完成正向传播 ############
    
    """ 
    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################
    d_cost_hyfs_input = hpfs_output - y
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 60000
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    
    ############ w1的权重误差计算 #######################
    ############ d_cost_w1 = {[(h-y)·w2] * hid_out*(1-hid_out)}·x #####################
    ############ d_cost_hid_out = (h-y)·w2  shape=m*393 #####################
    ############ 在使用 reLu的情况下， d_cost_hid
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T)
    #d_cost_hid_input = d_cost_hid_out*hid_output*(1-hid_output) ###隐藏层输入为sigmond的情况下 shape = 60000*393
    d_cost_hid_input = d_cost_hid_out*d_reLu(hid_output)         ####隐藏层输入为reLu的情况下 shape = 60000*393
    #print(d_cost_hid_input.shape)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    #print(d_cost_hid_input.shape)###### shape = 60000*392
    d_cost_w1 = np.matmul(x.T,d_cost_hid_input) / 60000
    ######## d_cost_w1.shape = 785*393 ############
    #print(d_cost_w1.shape)
    #print(d_cost_w1[0:20,:])
    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    """
    
    ####################### 下面是 用reLu 和 softmax 的情况 ###########################
    ####### dL/dwij = (h_outj -yj)*ai
    d_cost_hyfs_input = hpfs_output - y
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 60000  ##shape is 801*10
    #print(d_cost_w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T)   ### shape is m*801
    #print(d_cost_hid_out.shape)
    d_cost_hid_input = d_cost_hid_out*hid_output*(1-hid_output) ###隐藏层输入为sigmond的情况下 shape = 60000*801
    #d_cost_hid_input = d_cost_hid_out*d_reLu(hid_output) ### shape is m*801
    #print(d_cost_hid_input.shape)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1) ### shape is m*800
    #print(d_cost_hid_input.shape)
    d_cost_w1 = np.matmul(x.T,d_cost_hid_input) / 60000
    #print(d_cost_w1.shape)
    
    
    ####################### 下面是 用reLu 和 softmax 的情况 ###########################
    
    
    ######################## 上面代码完成反向传播计算 #########################
    ###########更新权重###################
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    np.savetxt("init_w1.txt",w1,delimiter=',')
    np.savetxt("init_w2.txt",w2,delimiter=',')
    
    if (i % 5 == 0) :
        print("J = ",J_softmax(hpfs_output,y))
        print(hpfs_output[0:1,:])
        print("-----------------count ",i,"------------------")
    #print("-----------------count ",i,"------------------")
    

J =  -120.81201951153574
[[1.00000000e+000 3.66275144e-048 4.14882046e-047 1.64601839e-104
  2.21850093e-045 3.23186851e-032 1.93788314e-025 4.21118188e-073
  8.01051249e-081 1.31643951e-070]]
-----------------count  0 ------------------
J =  -50.32004989186526
[[4.13959047e-23 3.29680534e-04 9.99670319e-01 1.19500197e-56
  1.72818330e-26 3.91019551e-21 5.54381398e-13 7.47808113e-25
  8.97217746e-34 2.83616623e-23]]
-----------------count  5 ------------------
J =  -30.56331569848179
[[3.93203426e-01 9.47240791e-11 1.61590922e-09 3.65559823e-30
  6.05444658e-01 4.29192731e-27 6.68255731e-22 7.06027979e-27
  4.23439433e-08 1.35187259e-03]]
-----------------count  10 ------------------
J =  -22.121491119161732
[[1.08129852e-01 8.79296045e-20 3.22729021e-20 1.37225834e-08
  2.86535132e-01 1.40896321e-07 6.05270035e-01 6.48263230e-05
  1.07616991e-19 2.21264979e-17]]
-----------------count  15 ------------------
J =  -17.951337713670046
[[2.12826211e-18 5.09888061e-06 2.54213903e-01 4.1012

KeyboardInterrupt: 

In [24]:
print(w1[0:10,0:5])
print(w2[0:10,0:5])

[[ 2.0908031  -0.03497779  3.64747814  2.1620542   0.19115367]
 [ 5.39666909  2.20596339 -2.75735997 -3.53002294  4.48736549]
 [ 2.88364245 -1.11049409  0.5470988   0.27539439  0.75385683]
 [-0.02236639  5.02631935  2.55262051  1.4060176   3.04603146]
 [-0.81975067  1.5868249   2.93002361  3.08238602 -0.57908652]
 [ 0.68481143  3.3779914   0.42109234 -0.21111829  1.03791403]
 [ 0.25351065  1.52636776  1.61062286  1.5458509   0.93691049]
 [ 2.91639746  3.59104517  6.23094289  2.13252639  2.59516306]
 [ 1.02614914  1.6984495   1.17329824  4.4166868  -4.07072622]
 [-2.06097082  2.73021364  3.90149049  0.73267352  1.95291059]]
[[-0.87350583  4.7756266   1.45660755  2.88027557  2.92056905]
 [ 3.29652349  1.82620259  0.29785105 -1.29956161  7.19833575]
 [ 2.9279532  -1.16479127 -0.81206219  1.48064105 -0.17223832]
 [ 1.11501254  0.42214961 -0.76819387  0.96081547  1.25056957]
 [ 5.22635844  3.49500883  1.30380087  1.35459019 -0.82949625]
 [ 0.94181011  1.51945949  2.28334858 -2.21658671  0.6

In [25]:
print(test_imgs[0].reshape(1,-1).shape)

(1, 784)


In [28]:
############################# 测试 ######################


for i in range(len(test_imgs)):
    x = test_imgs[2].reshape(1,-1) -255
    x = np.insert(x,0,1,axis=1)
    #print(x[0:10,:])
    hid_input = np.matmul(x,w1)
    hid_output = sigmond(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=1)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = softmax_vector(hpfs_input)
    print(test_labs[2])
    print(hpfs_output.reshape(-1,1))
    break

1
[[2.07280512e-02]
 [7.24841330e-32]
 [1.88521559e-04]
 [6.99710626e-22]
 [3.52415195e-16]
 [1.89711632e-11]
 [1.27278641e-02]
 [9.59506747e-29]
 [9.66355563e-01]
 [6.23558650e-12]]
